# Chess Logic W/JS
> Here's an example of my chess logic with Javascript

- layout: default
- badges: false
- toc: false
- categories: [week_8]

## Purpose
This blog is where I tested and created much of the chessboard logic that I used later in our chess project.

## Cell #1

This section creates the chessBoard object and also creates a printboard function that I use later for testing. It also creates this lettersOnBoard string which I use for calculations for moves. I use the index in the list for all of the calculations.

In [1]:
chessBoard = {};

lettersOnBoard = "abcdefgh";



function printBoard(){
    for (j = 0; j <= 7; j++){
        letter = lettersOnBoard[j];
        console.log("")
        for (i = 1; i <= 8; i++){
            var newKey = letter + i;
            process.stdout.write(chessBoard[newKey][0] + " ");
        }
    }
}




'abcdefgh'

## Cell #2
This cell defines the piece class. This class is intended to be a parent class for the other classes. Initially this class only had the constructor with position and color. Later however, I realized there were duplicate methods in the classes so I added them here. Those being the move method which actually changes the piece on the chessboard and getAvailableMoves which uses the captures and the freemoves and so is the same for all of the classes. 

In [2]:
for (j = 0; j <= 7; j++){
    letter = lettersOnBoard[j];
    for (i = 1; i <= 8; i++){
        var newKey = letter + i;
        chessBoard[newKey] = ["OO", undefined]
    } 
}


// piece class, to be extended by other classes
class piece{
    constructor(_position, _color){
        this.position = _position;
        this.color = _color
    }
    
    move(move, currentM){
        let currentBoard = chessBoard[currentM];
        if(this.getAvailableMoves().includes(move)) {
            this.position = move;
            chessBoard[move] = currentBoard;
            chessBoard[currentM] = ["OO", undefined];
        }
    }
    getAvailableMoves(){
        let freeMoves = this.getFreeMoves()
        let captures = this.getAvailableCaptures()
        captures.forEach((c) => {
            freeMoves.push(c);
        })
        return freeMoves;
    }
}


[ 'OO', undefined ]

## Cell #3
though this cell is higher up, this is actually the last peice I did and it was probably the easiest. I just copied the code from the bishop and rook and seperated them by adding a B and R at the end respectively and then combining them which methods without the identifier at the end in order to be used by the methods in the piece class. 

In [3]:
class queen extends piece{
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
        this.id = "Q";
        }   
    //method to return all of the available moves that the piece can make. 
    getTotalMovesR(){
        let currentPosition = this.position.split("");
        let moves = [];
        for (var i = 1; i <= 8; i++){
            var newMove = currentPosition[0] + i;
            moves[i - 1] = newMove;
        }
        for (var i = 1; i <= 8; i++){
            var newMove = lettersOnBoard[i - 1] + currentPosition[1];
            moves.push(newMove);
        }
        let totalMoves = [];
        moves.forEach((c) => {
            if (c != this.position){
                totalMoves.push(c);
            }
        });
        return totalMoves;
    }
    //method to return all of the obstructed moves based on the total moves
    getObstructedMovesR(){
        let totalMoves = this.getTotalMovesR();
        let obstructedMoves = [];
        let blockedMoves = [];
        let index = -1
        totalMoves.forEach((c) => {
            if (!obstructedMoves.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMoves.push(c);
                    index++
                    totalMoves.forEach((c) => {
                        try{
                            if (obstructedMoves[index][1] > this.position[1] && c[1] > obstructedMoves[index][1]){
                                blockedMoves.push(c)
                            }
                            else if (obstructedMoves[index][1] < this.position[1] && c[1] < obstructedMoves[index][1]){
                                blockedMoves.push(c)
                            }
                        } catch{}
                        try{
                            if (lettersOnBoard.indexOf(obstructedMoves[index][0]) > lettersOnBoard.indexOf(this.position[0]) && lettersOnBoard.indexOf(c[0]) > lettersOnBoard.indexOf(obstructedMoves[index][0])){
                                blockedMoves.push(c)
                            }
                            if (lettersOnBoard.indexOf(obstructedMoves[index][0]) < lettersOnBoard.indexOf(this.position[0]) && lettersOnBoard.indexOf(c[0]) < lettersOnBoard.indexOf(obstructedMoves[index][0])){
                                blockedMoves.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })
        blockedMoves.forEach((c) => {obstructedMoves.push(c);})
        return obstructedMoves;
    }
    //method to return all of the moves which are not obstructed
    getFreeMovesR(){
        let totalMoves = this.getTotalMovesR();
        let obstructedMoves = this.getObstructedMovesR();
        totalMoves = totalMoves.filter( (c) => !obstructedMoves.includes(c) );
        return totalMoves;
    }
    //method to return the pieces which can be captured. 
    getAvailableCapturesR(){
        // defines new variables as other methods in this class which may be useful.
        let totalMoves = this.getTotalMovesR();
        let obstructedMoves = this.getObstructedMovesR();
        // defines arrays
        let sameRow = [];
        let sameColumn = [];
        let columnNums = [];
        let columnDifs = [];
        let negDifsColumn = [];
        let posDifsColumn = [];
        let rowLets = [];
        let rowNums = [];
        let rowDifs = [];
        let posDifsRow = [];
        let negDifsRow = [];
        let captures = [];
        let finalCaptures = [];
        // finds all of the moves which are in the same row or in the same column as the rook.
        obstructedMoves.forEach((c) => {
            if (this.position.split("")[0] == c.split("")[0]){
                sameColumn.push(c);
            }
            else if (this.position.split("")[1] == c.split("")[1]){
                sameRow.push(c);
            }
        })
        //adds to a new array all of the numbers in the obstructed columns. Also converts it to an Integer
        sameColumn.forEach((c) => {
            columnNums.push(parseInt(c.split("")[1]));
        })        
        //elipses is a spread function, basically inputs each value in the array as it's own parameter. 
        //this foreach finds the difference between the pieces in the same column and the rooks current position
        columnNums.forEach((c) => {
            columnDifs.push(c - parseInt(this.position.split("")[1]))
        });
        //this foreach defines two new
        columnDifs.forEach((c) => {
            if (c < 0) negDifsColumn.push(Math.abs(c)); else posDifsColumn.push(c);
        })
        // finds the minimum positive number and the minimum negative number and converts them to an integer
        var posMinColumn = parseInt(Math.min(...posDifsColumn));
        var negMinColumn = parseInt(Math.min(...negDifsColumn));
        // works backwards to find the position on the board given the smallest differences 
        sameColumn.forEach((c) => {
            if (parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) + posMinColumn || parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) - negMinColumn){
                captures.push(c)
            }
        })
        // basically does all of the same stuff but for the rows using the index of the lettersOnBoard array
        sameRow.forEach((c) => {
            rowLets.push(c.split("")[0]);
        })
        rowLets.forEach((c) => {
            rowNums.push(lettersOnBoard.indexOf(c) + 1)
        })
        rowNums.forEach((c) => {
            rowDifs.push(parseInt(c) - (lettersOnBoard.indexOf(this.position.split("")[0]) + 1))
        })
        rowDifs.forEach((c) => {
            if (c < 0) negDifsRow.push(Math.abs(c)); else posDifsRow.push(c);
        })
        var posMinRow = parseInt(Math.min(...posDifsRow));
        var negMinRow = parseInt(Math.min(...negDifsRow))
        sameRow.forEach((c) => {
            if ((lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) + posMinRow + 1) || (lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) - negMinRow + 1)){
                captures.push(c)
            }
        })
        //checks if captures are the same color or not
        captures.forEach((c) => {
            if (chessBoard[c][0].split("")[0] != this.color){
                finalCaptures.push(c);
            }
        })
        return finalCaptures
    }
    getTotalMovesB(){
        let currentPosition = this.position;
        let movesLToR = [];
        let movesRToL = [];
        let furthestLeft = currentPosition;
        let furthestRight = currentPosition;
        let bruh = 0;
        while (furthestLeft[0] != "a" && furthestLeft[1] != 1){
            furthestLeft = lettersOnBoard[lettersOnBoard.indexOf(furthestLeft[0]) - 1] + (furthestLeft[1] - 1);
        }
        while (furthestRight[0] != "h" && furthestRight[1] != 1){
            furthestRight = lettersOnBoard[lettersOnBoard.indexOf(furthestRight[0]) + 1] + (furthestRight[1] - 1);
        }
        for (i = 0; i < 8 - lettersOnBoard.indexOf(furthestLeft[0]) - furthestLeft[1] + 1; i++){
            movesLToR.push(lettersOnBoard[lettersOnBoard.indexOf(furthestLeft[0]) + i] + (parseInt(furthestLeft[1]) + i));
        }
        for (i = 0; i < 2 + lettersOnBoard.indexOf(furthestRight[0]) - furthestRight[1]; i++){
            movesRToL.push(lettersOnBoard[lettersOnBoard.indexOf(furthestRight[0]) - i] + (parseInt(furthestRight[1]) + i));
        }
        let totalMovesLToR = [];
        let totalMovesRToL = []
        movesLToR.forEach((c) => {
            if (c != this.position){
                totalMovesLToR.push(c);
            }
        });
        movesRToL.forEach((c) => {
            if (c != this.position){
                totalMovesRToL.push(c);
            }
        });
        return [totalMovesLToR, totalMovesRToL];
    }
    //method to return all of the obstructed moves based on the total moves
    getObstructedMovesB(){
        let totalMoves = this.getTotalMovesB();
        let obstructedMovesLToR = [];
        let obstructedMovesRToL = [];
        let blockedMovesLToR = [];
        let blockedMovesRToL = [];
        let index = -1
        // Finds the moves which are behind an obstructed move and also finds all of the obstructed moves. Only for left to right. Does it by compating whether the letter + number is higher or lower. 
        totalMoves[0].forEach((c) => {
            if (!blockedMovesLToR.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMovesLToR.push(c);
                    index++
                    totalMoves[0].forEach((c) => {
                        try{
                            if (parseInt(obstructedMovesLToR[index][1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) > parseInt(this.position[1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) && parseInt(c[1]) + lettersOnBoard.indexOf(c[0]) > parseInt(obstructedMovesLToR[index][1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0])){
                                blockedMovesLToR.push(c)
                            }
                            else if (obstructedMovesLToR[index][1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) < this.position[1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) && c[1] + lettersOnBoard.indexOf(c[0]) < obstructedMovesLToR[index][1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0])){
                                blockedMovesLToR.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })
        index = -1
        // Finds the moves which are behind an obstructed move and also finds all of the obstructed moves. Only for right to left. Does it by finding whether the number is bigger or smaller (realized I was being dumb before but i'm not changing the old code. Because it's only one diaganol though you can easily find if its blocked just by the number.)
        totalMoves[1].forEach((c) => {
            if (!blockedMovesRToL.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMovesRToL.push(c);
                    index++
                    totalMoves[1].forEach((c) => {
                        try{
                            if (parseInt(c[1]) > parseInt(obstructedMovesRToL[index][1]) && parseInt(obstructedMovesRToL[index][1]) > parseInt(this.position[1])){
                                blockedMovesRToL.push(c)
                            }
                            else if (parseInt(c[1]) < parseInt(obstructedMovesRToL[index][1]) && parseInt(obstructedMovesRToL[index][1]) < parseInt(this.position[1])){
                                blockedMovesRToL.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })
        //seperates the obstructed moves and the blocked moves and returns both. 
        let obstructedMoves = [];
        obstructedMovesLToR.forEach((c) => [obstructedMoves.push(c)])
        obstructedMovesRToL.forEach((c) => [obstructedMoves.push(c)])
        let blockedMoves = [];
        blockedMovesLToR.forEach((c) => {blockedMoves.push(c);})
        blockedMovesRToL.forEach((c) => {blockedMoves.push(c);})
        obstructedMoves = obstructedMoves.filter((c) => !blockedMoves.includes(c))
        return [obstructedMoves, blockedMoves];
    }
    //method to return all of the moves which are not obstructed
    getFreeMovesB(){
        let totalMoves = this.getTotalMovesB()[0];
        this.getTotalMovesB()[1].forEach((c) => {totalMoves.push(c)})
        let obstructedMoves = this.getObstructedMovesB()[0];
        this.getObstructedMovesB()[1].forEach((c) => {obstructedMoves.push(c)})
        totalMoves = totalMoves.filter((c) => !obstructedMoves.includes(c) );
        return totalMoves;
    }
    //method to return the pieces which can be captured. 
    getAvailableCapturesB(){
        let finalCaptures = [];
        let obstructedMoves = this.getObstructedMovesB()[0]
        obstructedMoves.forEach((c) => {
            if (chessBoard[c][0][0] != this.color) {finalCaptures.push(c)}
        })
        console.log(finalCaptures)
        return finalCaptures
    }
    getFreeMoves(){
        let getFreeMovesB = this.getFreeMovesB()
        let getFreeMovesR = this.getFreeMovesR()
        let freeMoves = [];
        getFreeMovesB.forEach((c) => freeMoves.push(c))
        getFreeMovesR.forEach((c) => freeMoves.push(c))
        return freeMoves
    }
    getAvailableCaptures(){
        let getAvailableCapturesB = this.getAvailableCapturesB()
        let getAvailableCapturesR = this.getAvailableCapturesR()
        let captures = [];
        getAvailableCapturesB.forEach((c) => captures.push(c))
        getAvailableCapturesR.forEach((c) => captures.push(c))
        return captures
    }
}

let queen1 = new queen("d1", "w")
setBoard(queen1)

printMoves("d1")

function setBoard(obj){
    chessBoard[obj.position] = [obj.color + obj.id, obj]
}

function movePiece(currentM, newM){
    chessBoard[currentM][1].move(newM, currentM)
}

function printMoves(current){
    console.log(chessBoard[current][1].getAvailableMoves())
}



[]
[ 'e2',
  'f3',
  'g4',
  'h5',
  'c2',
  'b3',
  'a4',
  'd2',
  'd3',
  'd4',
  'd5',
  'd6',
  'd7',
  'd8',
  'a1',
  'b1',
  'c1',
  'e1',
  'f1',
  'g1',
  'h1' ]


## Cell #4
This is the bishop, the second to last piece I did. I though this piece was going to be really hard and I came up with complicated ways to figure out which moves are blocked and which moves are not. However it ended up working out really nicely. I was thinking about ways of coming up with the blocked moves without thinking of how to get all of the moves first. In the end to find all the moves I ended up going down and left until I reached an edge and then going from there and finding the rest of the moves by going back up and right. I did the same thing for the other way. This ended up with two seperate lists of moves. Then, since the lists were seperate, to check blocked moves I would only need to check one axis, not both. Because of this it was actually easy to find the blocked moves and it was similar to the rook. I actually made some advancements in this code and there's a lot of extra stuff I did for the rook that I didn't need to do.

In [4]:
class bishop extends piece{
    
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
        this.id = "B"
        }   
    //method to return all of the available moves that the piece can make. 
    getTotalMoves(){
        let currentPosition = this.position;
        let movesLToR = [];
        let movesRToL = [];
        let furthestLeft = currentPosition;
        let furthestRight = currentPosition;
        let bruh = 0;


        while (furthestLeft[0] != "a" && furthestLeft[1] != 1){
            furthestLeft = lettersOnBoard[lettersOnBoard.indexOf(furthestLeft[0]) - 1] + (furthestLeft[1] - 1);
        }

        while (furthestRight[0] != "h" && furthestRight[1] != 1){
            furthestRight = lettersOnBoard[lettersOnBoard.indexOf(furthestRight[0]) + 1] + (furthestRight[1] - 1);
        }

        for (i = 0; i < 8 - lettersOnBoard.indexOf(furthestLeft[0]) - furthestLeft[1] + 1; i++){
            movesLToR.push(lettersOnBoard[lettersOnBoard.indexOf(furthestLeft[0]) + i] + (parseInt(furthestLeft[1]) + i));
        }

        for (i = 0; i < 2 + lettersOnBoard.indexOf(furthestRight[0]) - furthestRight[1]; i++){
            movesRToL.push(lettersOnBoard[lettersOnBoard.indexOf(furthestRight[0]) - i] + (parseInt(furthestRight[1]) + i));
        }
        
        let totalMovesLToR = [];
        let totalMovesRToL = []

        movesLToR.forEach((c) => {
            if (c != this.position){
                totalMovesLToR.push(c);
            }
        });

        movesRToL.forEach((c) => {
            if (c != this.position){
                totalMovesRToL.push(c);
            }
        });

        return [totalMovesLToR, totalMovesRToL];
    }
    
    //method to return all of the obstructed moves based on the total moves
    getObstructedMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMovesLToR = [];
        let obstructedMovesRToL = [];
        let blockedMovesLToR = [];
        let blockedMovesRToL = [];
        let index = -1

        // Finds the moves which are behind an obstructed move and also finds all of the obstructed moves. Only for left to right. Does it by compating whether the letter + number is higher or lower. 
        totalMoves[0].forEach((c) => {
            if (!blockedMovesLToR.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMovesLToR.push(c);
                    index++
                    totalMoves[0].forEach((c) => {
                        try{
                            if (parseInt(obstructedMovesLToR[index][1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) > parseInt(this.position[1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) && parseInt(c[1]) + lettersOnBoard.indexOf(c[0]) > parseInt(obstructedMovesLToR[index][1]) + lettersOnBoard.indexOf(obstructedMovesLToR[index][0])){
                                blockedMovesLToR.push(c)
                            }
                            else if (obstructedMovesLToR[index][1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) < this.position[1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0]) && c[1] + lettersOnBoard.indexOf(c[0]) < obstructedMovesLToR[index][1] + lettersOnBoard.indexOf(obstructedMovesLToR[index][0])){
                                blockedMovesLToR.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })

        index = -1
        // Finds the moves which are behind an obstructed move and also finds all of the obstructed moves. Only for right to left. Does it by finding whether the number is bigger or smaller (realized I was being dumb before but i'm not changing the old code. Because it's only one diaganol though you can easily find if its blocked just by the number.)
        totalMoves[1].forEach((c) => {
            if (!blockedMovesRToL.includes(c)){
                if (chessBoard[c][0] != "OO"){
                    obstructedMovesRToL.push(c);
                    index++
                    totalMoves[1].forEach((c) => {
                        try{
                            if (parseInt(c[1]) > parseInt(obstructedMovesRToL[index][1]) && parseInt(obstructedMovesRToL[index][1]) > parseInt(this.position[1])){
                                blockedMovesRToL.push(c)
                            }
                            else if (parseInt(c[1]) < parseInt(obstructedMovesRToL[index][1]) && parseInt(obstructedMovesRToL[index][1]) < parseInt(this.position[1])){
                                blockedMovesRToL.push(c)
                            }
                        } catch{}
                    })
                } 
            }
        })

        //seperates the obstructed moves and the blocked moves and returns both. 
        let obstructedMoves = [];
        obstructedMovesLToR.forEach((c) => [obstructedMoves.push(c)])
        obstructedMovesRToL.forEach((c) => [obstructedMoves.push(c)])
        let blockedMoves = [];
        blockedMovesLToR.forEach((c) => {blockedMoves.push(c);})
        blockedMovesRToL.forEach((c) => {blockedMoves.push(c);})
        obstructedMoves = obstructedMoves.filter((c) => !blockedMoves.includes(c))
        return [obstructedMoves, blockedMoves];
    }

    //method to return all of the moves which are not obstructed
    getFreeMoves(){
        let totalMoves = this.getTotalMoves()[0];
        this.getTotalMoves()[1].forEach((c) => {totalMoves.push(c)})
        let obstructedMoves = this.getObstructedMoves()[0];
        this.getObstructedMoves()[1].forEach((c) => {obstructedMoves.push(c)})


        totalMoves = totalMoves.filter((c) => !obstructedMoves.includes(c) );

        return totalMoves;
    }

    //method to return the pieces which can be captured. 
    getAvailableCaptures(){
        let finalCaptures = [];
        let obstructedMoves = this.getObstructedMoves()[0]
        obstructedMoves.forEach((c) => {
            if (chessBoard[c][0][0] != this.color) {finalCaptures.push(c)}
        })
        console.log(finalCaptures)
        return finalCaptures
    }
}

let bishop1 = new bishop("d4", "w")
chessBoard[bishop1.position] = [bishop1.color + bishop1.id, bishop1]

let bishop2 = new bishop("e3", "b")
chessBoard[bishop2.position] = [bishop2.color + bishop2.id, bishop2]

let biship3 = new bishop("e5", "b")
chessBoard[biship3.position] = [biship3.color + biship3.id, biship3]

let bishop4 = new bishop("g1", "b")
chessBoard[bishop4.position] = [bishop4.color + bishop4.id, bishop4]

//printBoard();

bishop1.getAvailableMoves()
bishop1.getObstructedMoves()


[ 'e5', 'e3' ]


[ [ 'e5', 'e3' ], [ 'f6', 'g7', 'h8', 'g1', 'f2' ] ]

# Cell #5
This is for the pawn. This piece was easier than the rook and the bishop. The only complicated part was setting up double move for the first move, but it wasn't that bad. I set up a variable which displayed whether or not it had moved yet and then changed the available moves based on that. Also, this class has it's own unique move method which uses the parent class move and adds a single line where is changes the hasMoved variable to 1 to show that it can't move two anymore. The other thing that I had to do different was directionality, but I was able to just used the color to create a direction variable. The captures were also a little different because they weren't in the possible moves, but that actually made it easier. 

In [5]:
class pawn extends piece{
    constructor(_position, _color){
        // super is the position constructor, uh basically does some super cool inheritence stuff or something. 
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this pawn using the parent method
        if (_color == "w"){this.direction = 1}
        else if (_color == "b") {this.direction = -1}
        this.hasMoved = 0;
        this.id = "P";
    }
    move(move, currentM){
        super.move(move, currentM)
        this.hasMoved = 1
    }
    getTotalMoves(){
        let moves = [];
        let currentPosition = this.position.split("");

        if(this.hasMoved == 0){
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (1 * this.direction)))
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (2 * this.direction)))
        }
        else{
            moves.push(currentPosition[0] + (parseInt(currentPosition[1]) + (1 * this.direction)))
        }
        return moves;
    }
    getFreeMoves(){
        let moves = this.getTotalMoves();
        let freeMoves = [];

        if (chessBoard[moves[0]][0] == "OO"){
            freeMoves.push(moves[0]);
            try{
                if (chessBoard[moves[1]][0] == "OO"){freeMoves.push(moves[1]);}
            }catch{}
        }
        return freeMoves;
    } 
    getAvailableCaptures(){
        let captures = [];
        let currentPosition = this.position.split(""); 
        let possibleLets = [
            lettersOnBoard[lettersOnBoard.indexOf(currentPosition[0]) - 1],
            lettersOnBoard[lettersOnBoard.indexOf(currentPosition[0]) + 1]
        ];
        possibleLets = possibleLets.filter(c => c != undefined);
        possibleLets.forEach((c) => {
            let a = parseInt(currentPosition[1]) + (1 * this.direction)
            let check = c + a
            if (chessBoard[check][0][0] != this.color && chessBoard[check][0] != "OO"){
                    captures.push(check);
            }
        })
        return captures;
    }
}

let pawn1 = new pawn("c5", "b")
chessBoard[pawn1.position] = [pawn1.color + pawn1.id, pawn1]

printBoard()




OO OO OO OO OO OO OO OO 
OO OO OO OO OO OO OO OO 
OO OO OO OO bP OO OO OO 
wQ OO OO wB OO OO OO OO 
OO OO bB OO bB OO OO OO 
OO OO OO OO OO OO OO OO 
bB OO OO OO OO OO OO OO 
OO OO OO OO OO OO OO OO 

In [6]:
class king extends piece{
    constructor(_position, _color){
        super(_position, _color);
        // automatically sets the spot on the board which is passed in to this rook
    }
    //method to return all of the available moves that the piece can make. 
    getTotalMoves(){
        let currentPosition = this.position.split("");
        let moves = [];

        let aboveNum = parseInt(currentPosition[1]) + 1
        let currentLetNum = lettersOnBoard.indexOf(currentPosition[0])
        
        for (var i = -1; i <= 1; i++){
            moves.push(lettersOnBoard[currentLetNum + i] + aboveNum);
        }
    }
    let belowNum = parseInt(currentPosition[1]) - 1
    for (var i = -1; i <= 1; i++){
        if ((belowNum != 0) && ((currentLetNum + i) != 8) && (currentLetNum + i) != -1){
            moves.push(lettersOnBoard[currentLetNum + i] + belowNum);
        }
        
        let sameNum = parseInt(currentPosition[1])

        for (var i = -1; i <= 1;i++){
            if (i != 0){
                moves.push(lettersOnBoard[currentLetNum + i] + sameNum);
            }
        }

        return moves;
    }

    //method to return all of the obstructed moves based on the total moves
    getObstructedMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = [];
        for (var i = 0; i < totalMoves.length; i++){
            if (chessBoard[totalMoves[i]][0] != "OO"){
                obstructedMoves.push(totalMoves[i]);
            }
        }
        return obstructedMoves;
    }

    //method to return all of the moves which are not obstructed
    getFreeMoves(){
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();

        totalMoves.forEach((c) => {
            //console.log(c);
            for (var i = 0; i < obstructedMoves.length; i ++){
                if (c == obstructedMoves[i]){
                    totalMoves.splice(totalMoves.indexOf(c), 1);
                }
            }
        })

        return totalMoves;
    }
    

    //method to return the pieces which can be captured. 
    getAvailableCaptures(){
        // defines new variables as other methods in this class which may be useful.
        let totalMoves = this.getTotalMoves();
        let obstructedMoves = this.getObstructedMoves();
        // defines arrays
        let sameRow = [];
        let sameColumn = [];
        let columnNums = [];
        let columnDifs = [];
        let negDifsColumn = [];
        let posDifsColumn = [];
        let rowLets = [];
        let rowNums = [];
        let rowDifs = [];
        let posDifsRow = [];
        let negDifsRow = [];
        let captures = [];
        let finalCaptures = [];
        // finds all of the moves which are in the same row or in the same column as the rook.
        obstructedMoves.forEach((c) => {
            if (this.position.split("")[0] == c.split("")[0]){
                sameColumn.push(c);
            }
            else if (this.position.split("")[1] == c.split("")[1]){
                sameRow.push(c);
            }
        })
        //adds to a new array all of the numbers in the obstructed columns. Also converts it to an Integer
        sameColumn.forEach((c) => {
            columnNums.push(parseInt(c.split("")[1]));
        })        
        //elipses is a spread function, basically inputs each value in the array as it's own parameter. 
        //this foreach finds the difference between the pieces in the same column and the rooks current position
        columnNums.forEach((c) => {columnDifs.push(c - parseInt(this.position.split("")[1]))});
        //this foreach defines two new
        columnDifs.forEach((c) => {if (c < 0) negDifsColumn.push(Math.abs(c)); else posDifsColumn.push(c);})
        // finds the minimum positive number and the minimum negative number and converts them to an integer
        var posMinColumn = parseInt(Math.min(...posDifsColumn));
        var negMinColumn = parseInt(Math.min(...negDifsColumn));
        // works backwards to find the position on the board given the smallest differences 
        sameColumn.forEach((c) => {if (parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) + posMinColumn || parseInt(c.split("")[1]) == parseInt(this.position.split("")[1]) - negMinColumn){captures.push(c)}})
        // basically does all of the same stuff but for the rows using the index of the lettersOnBoard array
        sameRow.forEach((c) => {rowLets.push(c.split("")[0]);})
        rowLets.forEach((c) => {rowNums.push(lettersOnBoard.indexOf(c) + 1)})
        rowNums.forEach((c) => {rowDifs.push(parseInt(c) - (lettersOnBoard.indexOf(this.position.split("")[0]) + 1))})
        rowDifs.forEach((c) => {if (c < 0) negDifsRow.push(Math.abs(c)); else posDifsRow.push(c);})
        var posMinRow = parseInt(Math.min(...posDifsRow));
        var negMinRow = parseInt(Math.min(...negDifsRow))
        sameRow.forEach((c) => {if ((lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) + posMinRow + 1) || (lettersOnBoard.indexOf(c.split("")[0]) + 1) == (lettersOnBoard.indexOf(this.position.split("")[0]) - negMinRow + 1)){captures.push(c)}})
        //checks if captures are the same color or not
        captures.forEach((c) => {
            if (chessBoard[c].split("")[0] != this.color){
                finalCaptures.push(c);
            }
        })
        return finalCaptures
    }
    
}

let king1 = new king("d5", "b");

console.log(king1.getObstructedMoves())


[ 'd2', 'e2', 'f2', 'd1', 'f1' ]


[]

In [7]:
let rookb1 = new rook("a8", "b");
setBoard(rookb1)

let rookb2 = new rook("h8", "b");
setBoard(rookb2)

let rookw1 = new rook("a1", "w");
setBoard(rookw1)

let rookw2 = new rook("h1", "w");
setBoard(rookw2)

let pawnw1 = new pawn("a2", "w")
setBoard(pawnw1)

let pawnw2 = new pawn("b2", "w")
setBoard(pawnw2)

let pawnw3 = new pawn("c2", "w")
setBoard(pawnw3)

let pawnw4 = new pawn("d2", "w")
setBoard(pawnw4)

let pawnw5 = new pawn("e2", "w")
setBoard(pawnw5)

let pawnw6 = new pawn("f2", "w")
setBoard(pawnw6)

let pawnw7 = new pawn("g2", "w")
setBoard(pawnw7)

let pawnw8 = new pawn("h2", "w")
setBoard(pawnw8)

printBoard()
movePiece("g2", "g4");
movePiece("a1", "g1");
console.log("");
movePiece("g4", "g5");
movePiece("g5", "g6");
movePiece("g6", "g7");
printMoves("g7")
movePiece("g7", "h8");
printBoard();





function movePiece(currentM, newM){
    chessBoard[currentM][1].move(newM, currentM)
}

function printMoves(current){
    console.log(chessBoard[current][1].getAvailableMoves())
}

ReferenceError: rook is not defined